### Scripts to compute POWER (baseline, windows, ST plots)
#### Scripts to select COMMON electrodes and ROIs

In [2]:
import numpy as np
from os import path, listdir, makedirs
from itertools import product
from brainpipe.system import study

In [20]:
"""
Compute plots of power for all frequency bands in single electrodes
Baseline and Odor conditions
"""
import matplotlib.pyplot as plt 
from brainpipe.visual import addLines, BorderPlot, rmaxis

st = study('Olfacto')
path_data = path.join(st.path, 'feature/')
subjects = ['LEFC','VACJ','SEMC','LEFC','PIRJ']
conds = ['low', 'high']
reps = ['_Encoding_EpiPerf_LowHigh_olfactory/','_Retrieval_EpiPerf_LowHigh_olfactory/']
filename = '{}{}_odor_{}_common_renamed_olfactory_pow.npz'
savename = path.join(path_data,'{}Avg_plots/{}_{}_A_{}_elec({}).png')

def plots_avg_power(rep,su,freq,labels,time,pow1_f,pow2_f):
    for elec in range(labels.shape[0]):
        x = np.concatenate((pow1_f[elec],pow2_f[elec]),axis=-1)
        y = np.array([1]*pow1_f.shape[-1] + [0]*pow2_f.shape[-1])
        BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=['blue','black'],linewidth=2, 
                  ncol=1, xlabel='Time (s)',ylabel = r'Power', legend=['odor','bsl'])
        rmaxis(plt.gca(), ['right', 'top'])
        addLines(plt.gca(), vLines=[0], vColor=['darkgray'], vWidth=[2])
        plt.legend(loc=0, handletextpad=0.1, frameon=False)
        
        title = su+' avg '+freq+' - all trials for elec('+str(elec)+') '+labels[elec]
        plt.title(title)
        ###############################################################################
        if not path.exists(path_data+rep+'Avg_plots/'):
            makedirs(path_data+rep+'Avg_plots/')
        ############################################################################### 
        #fig.savefig(path2save+su+'_'+freq+'_ST_'+labels[elec]+'_elec('+str(elec)+').pdf')
        plt.savefig(savename.format(rep,su,freq,labels[elec],str(elec)))
        plt.clf()
        plt.close()
    
for su,rep in product(subjects,reps): 
    mat1 = np.load(path_data+filename.format(rep,su,conds[0]),allow_pickle=True)
    mat2 = np.load(path_data+filename.format(rep,su,conds[1]),allow_pickle=True)
    labels, fnames = mat1['labels'], mat1['fnames']
    time, pow1, pow2 = mat1['time'], mat1['xpow_od'], mat2['xpow_od']
    bsl1, bsl2 = mat1['xpow_bsl'], mat2['xpow_bsl']
    pow_od = np.concatenate((pow1,pow2),axis=-1)
    pow_bsl = np.concatenate((bsl1,bsl2),axis=-1)
    for i,freq in enumerate(fnames):
        pow_od_f, pow_bsl_f = pow_od[i,...], pow_bsl[i,...]
        plots_avg_power(rep,su,freq,labels,time,pow_od_f,pow_bsl_f)

-> Olfacto loaded


In [ ]:
"""
Compute single-trial plots of power for all frequency bands in single electrodes
using power corrected by the baseline 'pow_n'
"""
import matplotlib.pyplot as plt 
from brainpipe.visual import addLines

st = study('Olfacto')
path_data = path.join(st.path, 'feature/')
subjects = ['LEFC','VACJ','SEMC','LEFC','PIRJ']
conds = ['low', 'high']
reps = ['_Encoding_EpiPerf_LowHigh_olfactory/','_Retrieval_EpiPerf_LowHigh_olfactory/']
filename = '{}{}_odor_{}_common_renamed_olfactory_pow.npz'
savename = path.join(path_data,'{}ST_plots/{}_{}_ST_{}_elec({}).png')

def plots_st_power(rep,su,freq,labels,time,pow1_f,pow2_f):
    data_all = np.concatenate((pow1_f,pow2_f),axis=-1)
    for elec in range(labels.shape[0]):
        x = data_all[elec].T
        f_max = np.array([np.argmax(trial) for trial in x])
        arr1inds = f_max.argsort()
        sorted_x = x[arr1inds[::-1]]
        fig = plt.figure(figsize=(5,4))
        title = su+' ST '+freq+' - '+conds[0].capitalize()+' & '+conds[1].capitalize()+\
            ' elec('+str(elec)+') '+labels[elec]
        plt.imshow(sorted_x, cmap='jet',interpolation='none', origin ='lower', 
                   aspect='auto',vmin=0, vmax=np.max(x), extent=[-0.5,2,0,x.shape[0]])
        plt.title(title)
        plt.ylabel('Trials')
        plt.xlabel('Time (s)')
        addLines(plt.gca(),vLines=[0], vColor=['red'], vShape=['--'], vWidth=[1.5])
        cb = plt.colorbar(ticks=[0,np.max(x)/2,np.max(x)],label='Power')
        ###############################################################################
        if not path.exists(path_data+rep+'ST_plots/'):
            makedirs(path_data+rep+'ST_plots/')
        ############################################################################### 
        #fig.savefig(path2save+su+'_'+freq+'_ST_'+labels[elec]+'_elec('+str(elec)+').pdf')
        fig.savefig(savename.format(rep,su,freq,labels[elec],str(elec)))
        plt.clf()
        plt.close()
    
for su,rep in product(subjects,reps): 
    mat1 = np.load(path_data+filename.format(rep,su,conds[0]),allow_pickle=True)
    mat2 = np.load(path_data+filename.format(rep,su,conds[1]),allow_pickle=True)
    labels, fnames = mat1['labels'], mat1['fnames']
    time, pow1, pow2 = mat1['time_n'], mat1['xpow_n'], mat2['xpow_n']
    for i,freq in enumerate(fnames):
        pow1_f, pow2_f = pow1[i,...], pow2[i,...]
        plots_st_power(rep,su,freq,labels,time,pow1_f,pow2_f)

In [4]:
"""
Compute POWER (nfreqs x nelecs x npts x ntrials)
"""
from brainpipe.feature import power
from brainpipe.feat.utils._feat import _manageWindow

st = study('Olfacto')
data_path = path.join(st.path, 'database/{}/')
dataname = '{}_odor_{}_common_renamed_OFC_HC.npz'
save_path = path.join(st.path,'feature/_{}_mean_3s_bsl/')
savename = dataname.replace('.npz','_pow.npz')
reps = ['Encoding_EpiPerf_LowHigh','Retrieval_EpiPerf_LowHigh']
conds = ['high','low']
subjects = ['LEFC','SEMC','VACJ','PIRJ','FERJ','CHAF']

################################################################################################
# Define POWER SETTINGS
bsl, width, step = None, 358, 51
fnames = ['theta', 'alpha', 'beta', 'gamma']
f_lim = [[3,7], [8,13], [13,30], [30,120]]
f_splits = [None,None,None,9]
################################################################################################

def compute_power(su,rep,cond):
    mat = np.load(data_path.format(rep)+dataname.format(su,cond),allow_pickle=True)
    x, labels, xyz, chans = mat['x'], mat['labels'], mat['xyz'], mat['channels']
    n_elec, n_pts, n_trials = x.shape
    print ('--> compute power on : ', dataname.format(su,cond),'shape', x.shape)
    bsl_ = (768,1024) #[-1000ms to -300ms]
    odor_ = [1024,2551] #[-800 to 2s] following inspiration onset
    odor_norm = [1024,2551] #[-500 to +2000ms]

    #compute power for the whole time series and then cut (avoid edge effect)
    powObj_x = power(512, n_pts, baseline=None, method='hilbert1',
                     f=f_lim, width=width, step=step, split=f_splits)
    powObj_x2 = power(512, n_pts, baseline=bsl_, norm=4, method='hilbert1',
                 f=f_lim, width=width, step=step, split=f_splits)
    pow_all, _ = powObj_x.get(x,n_jobs=-1)
    pow_norm, _ = powObj_x2.get(x,n_jobs=-1)
    win_all, _ = _manageWindow(x.shape[1], width=width, step=step)
    id_bsl = [i for i,win in enumerate(win_all) if win[0] == bsl_[0]]
    id_od = [i for i,win in enumerate(win_all) if (win[0] >= odor_[0] \
                                                   and win[1] <= odor_[1])]
    id_norm = [i for i,win in enumerate(win_all) if (win[0] >= odor_norm[0]\
                                                     and win[1] <= odor_norm[1])]
    pow_bsl = pow_all[:,:,id_bsl,:]
    pow_bsl = np.repeat(pow_bsl, len(id_od), axis=2)
    pow_odor = pow_all[:,:,id_od,:]
    pow_od_norm = pow_norm[:,:,id_norm,:]
    time = np.arange(-1,2,3/len(id_od))
    #save power data
    if not path.exists(save_path.format(rep)):
        makedirs(save_path.format(rep))
    np.savez(save_path.format(rep)+savename.format(su,cond), xpow_od=pow_odor,
        xpow_bsl=pow_bsl, xpow_n=pow_od_norm,f=f_lim, split=f_splits, width=width, 
        step=step,fnames=fnames, time=time, labels=labels, xyz=xyz, 
        channels=chans)

for su,rep,cond in product(subjects,reps,conds):
    compute_power(su,rep,cond)

-> Olfacto loaded
--> compute power on :  LEFC_odor_high_common_renamed_OFC_HC.npz shape (36, 3584, 21)
--> compute power on :  LEFC_odor_low_common_renamed_OFC_HC.npz shape (36, 3584, 34)
--> compute power on :  LEFC_odor_high_common_renamed_OFC_HC.npz shape (36, 3584, 12)
--> compute power on :  LEFC_odor_low_common_renamed_OFC_HC.npz shape (36, 3584, 12)
--> compute power on :  SEMC_odor_high_common_renamed_OFC_HC.npz shape (62, 3584, 9)
--> compute power on :  SEMC_odor_low_common_renamed_OFC_HC.npz shape (62, 3584, 31)
--> compute power on :  SEMC_odor_high_common_renamed_OFC_HC.npz shape (62, 3584, 9)
--> compute power on :  SEMC_odor_low_common_renamed_OFC_HC.npz shape (62, 3584, 15)
--> compute power on :  VACJ_odor_high_common_renamed_OFC_HC.npz shape (48, 3584, 11)
--> compute power on :  VACJ_odor_low_common_renamed_OFC_HC.npz shape (48, 3584, 11)
--> compute power on :  VACJ_odor_high_common_renamed_OFC_HC.npz shape (48, 3584, 12)
--> compute power on :  VACJ_odor_low_commo

In [ ]:
"""
Select ONLY specific ROIs in data
"""

st = study('Olfacto')
data_path = path.join(st.path, 'database/{}/')
dataname = '{}_odor_{}_common_renamed_OFC_HC.npz'
savename = '{}_odor_{}_common_renamed_olfactory.npz'
reps = ['Encoding_EpiPerf_LowHigh','Retrieval_EpiPerf_LowHigh']
conds = ['high','low']
subjects = ['LEFC','SEMC','VACJ','PIRJ','FERJ']
rois_ = ['Amg','pPirT','OFC_olf','aHC','pHC']

def save_selected_rois(rep,su,cond):
    mat = np.load(data_path.format(rep)+dataname.format(su,cond), allow_pickle=True)
    labels = mat['labels']
    #create boolean for electrodes in specific ROIs
    id_sel = [i for i,lab in enumerate(labels) if lab in rois_]
    xyz = mat['xyz'][id_sel]
    
    #select specific ROIs in data
    xyz, data = mat['xyz'][id_sel], mat['x'][id_sel]
    labels_n, chan = labels[id_sel], mat['channels'][id_sel]
    #save new information
    np.savez(data_path.format(rep)+savename.format(su,cond),
            x=data, xyz=xyz, labels=labels_n, channels=chan)

for su, cond, rep in product(subjects,conds,reps):
    save_selected_rois(rep,su,cond)

In [6]:
"""
BY CONDITION HIGH LOW // BY ODOR (change the repos)
1- Select COMMON electrodes btw E and R
2- Rename electrodes in OFC and HC
"""
from os import listdir
st = study('Olfacto')
data_path = path.join(st.path, 'database/{}/')
dataname = '{}_odor_{}_bipo_sel_physFT.npz'
savename = '{}_odor_{}_common_renamed_OFC_HC.npz'
reps = ['Encoding_By_Odor','Retrieval_By_Odor']
conds = ['high','low']
subjects = ['LEFC','SEMC','VACJ','PIRJ','FERJ','CHAF']

x0,y0,z0 = 20, 30,-16
x1,y1,z1 = -20, 32,-16
rad = 11

def rename_labels(X,xyz,labels):
    new_lab = []
    for e in range(X.shape[0]):
        x,y,z = xyz[e][0], xyz[e][1], xyz[e][2]
        label = labels[e]
        id_R = 1 if all([x0-rad<=x<=x0+rad, y0-rad<=y<=y0+rad,
                    z0-rad<=z<=z0+rad]) else 0
        id_L = 1 if all([x1-rad<=x<=x1+rad, y1-rad<=y<=y1+rad,
                    z1-rad<=z<=z1+rad]) else 0
        idx = id_R + id_L
        if label == 'HC' and y >= -26 :
            new_lab.append('aHC')
        elif label == 'pPirT-Amg':
            new_lab.append('pPirT')
        elif label == 'HC' and y < -26 :
            new_lab.append('pHC')
        elif idx == 1 :
            new_lab.append('OFC_olf')
        else:
            new_lab.append(label)
    return np.array(new_lab)

def save_common_elecs(f):
    su = f.split('_')[0]
    if su != 'MICP':
        #load data matrices for E and R
        mat_t0 = np.load(data_path.format(reps[0])+f, allow_pickle=True)
        mat_t1 = np.load(data_path.format(reps[1])+f, allow_pickle=True)
        chan0, chan1 = mat_t0['channels'], mat_t1['channels']
        #create boolean for electrodes present both at E and R
        id_t0 = [i for i,c in enumerate(chan0) if c in chan1]
        id_t1 = [i for i,c in enumerate(chan1) if c in chan0]
        #select common E and R data
        xyz_t0, data_t0 = mat_t0['xyz'][id_t0], mat_t0['x'][id_t0]
        labels_t0, chan_t0 = mat_t0['Mai_RL'][id_t0], chan0[id_t0]
        xyz_t1, data_t1 = mat_t1['xyz'][id_t1], mat_t1['x'][id_t1]
        labels_t1, chan_t1 = mat_t1['Mai_RL'][id_t1], chan1[id_t1]
        #rename electrodes
        new_labels_t0 = rename_labels(data_t0,xyz_t0,labels_t0)
        new_labels_t1 = rename_labels(data_t1,xyz_t1,labels_t1)
        #save new information
        print(f, data_t0.shape, data_t1.shape)
        np.savez(data_path.format(reps[0])+f.replace('bipo_sel_physFT','common_renamed_OFC_HC'),
            x=data_t0, xyz=xyz_t0, labels=new_labels_t0, channels=chan_t0)
        np.savez(data_path.format(reps[1])+f.replace('bipo_sel_physFT','common_renamed_OFC_HC'),
                x=data_t1, xyz=xyz_t1, labels=new_labels_t1, channels=chan_t1)

files_E = listdir(data_path.format(reps[0]))
for f in files_E:
    if f.endswith('_bipo_sel_physFT.npz'):
        save_common_elecs(f)

-> Olfacto loaded
CHAF_odor_1_bipo_sel_physFT.npz (57, 3584, 1) (57, 3584, 3)
CHAF_odor_2_bipo_sel_physFT.npz (57, 3584, 3) (57, 3584, 3)
CHAF_odor_3_bipo_sel_physFT.npz (57, 3584, 3) (57, 3584, 3)
LEFC_odor_14_bipo_sel_physFT.npz (36, 3584, 6) (36, 3584, 3)
LEFC_odor_15_bipo_sel_physFT.npz (36, 3584, 11) (36, 3584, 3)
LEFC_odor_16_bipo_sel_physFT.npz (36, 3584, 16) (36, 3584, 3)
LEFC_odor_17_bipo_sel_physFT.npz (36, 3584, 7) (36, 3584, 3)
PIRJ_odor_9_bipo_sel_physFT.npz (30, 3584, 6) (30, 3584, 4)
SEMC_odor_10_bipo_sel_physFT.npz (62, 3584, 7) (62, 3584, 3)
SEMC_odor_11_bipo_sel_physFT.npz (62, 3584, 4) (62, 3584, 3)
SEMC_odor_12_bipo_sel_physFT.npz (62, 3584, 6) (62, 3584, 3)
FERJ_odor_12_bipo_sel_physFT.npz (45, 3584, 4) (45, 3584, 3)
FERJ_odor_13_bipo_sel_physFT.npz (45, 3584, 3) (45, 3584, 3)
FERJ_odor_16_bipo_sel_physFT.npz (45, 3584, 3) (45, 3584, 3)
SEMC_odor_9_bipo_sel_physFT.npz (62, 3584, 3) (62, 3584, 3)
VACJ_odor_10_bipo_sel_physFT.npz (48, 3584, 3) (48, 3584, 3)
VACJ_odor